# Proyecto Final: Clasificador de Documentos con IA
## Maestría en IoT y AI - Materia de AI

### Descripción del Proyecto
Este notebook implementa un sistema de clasificación de documentos usando técnicas de NLP y Machine Learning.
El sistema puede clasificar documentos en tres categorías: emails, resumes (CVs) y publicaciones científicas.

### Pipeline del Proyecto:
1. Conversión de formatos de imagen (TIF/PDF a PNG)
2. Extracción de texto mediante OCR (Tesseract)
3. Análisis exploratorio exhaustivo de datos
4. División estratificada de datos (70% train, 20% validation, 10% test)
5. Entrenamiento de modelos con validación cruzada
6. Evaluación y selección del mejor modelo

## 1. Instalación de Dependencias

Instalamos las librerías necesarias para el procesamiento de imágenes, OCR, NLP y ML.

In [ ]:
# Descomentar para instalar dependencias necesarias
# %pip install nltk pytesseract scikit-learn pillow pdf2image matplotlib seaborn wordcloud xgboost lightgbm imbalanced-learn

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LEONI\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\LEONI\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LEONI\AppData\Roaming\nltk_data...


## 2. Importación de Librerías

Organizamos las importaciones por funcionalidad para mejor legibilidad y mantenimiento.

In [ ]:
# ============================================================================
# IMPORTACIONES DE LIBRERÍAS
# ============================================================================

# --- Librerías base de Python ---
import os
import re
import pickle
import warnings
from string import punctuation
from glob import glob

# --- Procesamiento de datos ---
import pandas as pd
import numpy as np

# --- Procesamiento de imágenes ---
from PIL import Image
import pytesseract

# --- Visualización ---
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# --- Procesamiento de lenguaje natural (NLP) ---
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# --- Machine Learning: Preprocesamiento ---
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA

# --- Machine Learning: Modelos ---
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# --- Machine Learning: Evaluación ---
from sklearn.metrics import (
    classification_report, confusion_matrix, accuracy_score,
    precision_recall_fscore_support, roc_auc_score, roc_curve
)

# Configuración general
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Librerías importadas correctamente")

In [ ]:
# ============================================================================
# DESCARGA DE RECURSOS NLTK
# ============================================================================

# Descargamos los recursos necesarios de NLTK para procesamiento de texto
try:
    nltk.download('stopwords', quiet=True)
    nltk.download('punkt', quiet=True)
    nltk.download('punkt_tab', quiet=True)
    nltk.download('wordnet', quiet=True)
    nltk.download('omw-1.4', quiet=True)
    print("✓ Recursos NLTK descargados correctamente")
except Exception as e:
    print(f"⚠ Error descargando recursos NLTK: {e}")

# Cargamos la lista de stopwords en inglés
# Estas palabras comunes serán eliminadas durante el preprocesamiento
stopwords_list = stopwords.words("english")
print(f"✓ Stopwords cargadas: {len(stopwords_list)} palabras")

In [ ]:
# ============================================================================
# CONFIGURACIÓN DE TESSERACT OCR
# ============================================================================

# Configuramos la ruta al ejecutable de Tesseract OCR
# NOTA: Ajustar esta ruta según la instalación en tu sistema
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Verificamos que Tesseract esté instalado correctamente
try:
    version = pytesseract.get_tesseract_version()
    print(f"✓ Tesseract OCR versión {version} configurado correctamente")
except Exception as e:
    print(f"⚠ Error: Tesseract no encontrado. Por favor instalar desde: https://github.com/UB-Mannheim/tesseract/wiki")
    print(f"   Detalles del error: {e}")

## 3. PASO 1a: Conversión de Archivos TIF a PNG

Función para convertir imágenes en formato TIF a PNG, necesario para estandarizar el formato de entrada.

In [ ]:
# ============================================================================
# FUNCIONES DE CONVERSIÓN DE FORMATO DE IMAGEN
# ============================================================================

def convert_tif_to_png(input_folder, output_folder=None, delete_original=False):
    """
    Convierte archivos .TIF/.TIFF a formato .PNG
    
    Parámetros:
    -----------
    input_folder : str
        Ruta de la carpeta que contiene archivos TIF
    output_folder : str, opcional
        Ruta de la carpeta de salida. Si es None, se usa la misma carpeta
    delete_original : bool, opcional
        Si True, elimina los archivos TIF originales después de convertir
    
    Retorna:
    --------
    int : Número de archivos convertidos exitosamente
    
    Justificación:
    --------------
    - PNG es un formato sin pérdida de calidad y ampliamente compatible
    - Facilita el procesamiento uniforme de todas las imágenes
    - Reduce tamaño de archivo comparado con TIF sin comprimir
    """
    
    if output_folder is None:
        output_folder = input_folder
    
    # Crear carpeta de salida si no existe
    os.makedirs(output_folder, exist_ok=True)
    
    # Buscar todos los archivos TIF/TIFF
    tif_files = glob(os.path.join(input_folder, "*.tif")) + \
                glob(os.path.join(input_folder, "*.tiff")) + \
                glob(os.path.join(input_folder, "*.TIF")) + \
                glob(os.path.join(input_folder, "*.TIFF"))
    
    converted_count = 0
    
    for tif_path in tif_files:
        try:
            # Abrir imagen TIF
            img = Image.open(tif_path)
            
            # Generar nombre del archivo PNG
            filename = os.path.basename(tif_path)
            png_filename = os.path.splitext(filename)[0] + '.png'
            png_path = os.path.join(output_folder, png_filename)
            
            # Convertir y guardar como PNG
            img.save(png_path, 'PNG')
            converted_count += 1
            
            # Eliminar original si se solicita
            if delete_original:
                os.remove(tif_path)
            
            print(f"  ✓ Convertido: {filename} → {png_filename}")
            
        except Exception as e:
            print(f"  ✗ Error convirtiendo {filename}: {e}")
    
    print(f"\n✓ Conversión completada: {converted_count} archivos convertidos")
    return converted_count


# Ejemplo de uso (comentado - descomentar para usar):
# convert_tif_to_png(r"datasets\mi_carpeta_tif", r"datasets\mi_carpeta_png")

## 4. PASO 1b: Conversión de Archivos PDF a PNG

Función para convertir documentos PDF a imágenes PNG, extrayendo cada página como imagen individual.

In [ ]:
def convert_pdf_to_png(input_folder, output_folder=None, dpi=200):
    """
    Convierte archivos PDF a imágenes PNG (una imagen por página)
    
    Parámetros:
    -----------
    input_folder : str
        Ruta de la carpeta que contiene archivos PDF
    output_folder : str, opcional
        Ruta de la carpeta de salida. Si es None, se crea subcarpeta 'png_output'
    dpi : int, opcional
        Resolución de la imagen de salida (default: 200)
        Mayor DPI = mejor calidad pero archivos más grandes
    
    Retorna:
    --------
    int : Número total de páginas convertidas
    
    Justificación del DPI:
    ----------------------
    - 200 DPI: Balance óptimo entre calidad OCR y tamaño de archivo
    - Tesseract OCR funciona eficientemente entre 150-300 DPI
    - DPI muy alto (>300) aumenta tiempo de procesamiento sin mejora significativa
    
    Nota:
    -----
    Requiere instalar: pip install pdf2image
    En Windows también requiere: poppler (descargar desde https://github.com/oschwartz10612/poppler-windows/releases/)
    """
    
    try:
        from pdf2image import convert_from_path
    except ImportError:
        print("⚠ Error: pdf2image no está instalado.")
        print("  Instalar con: pip install pdf2image")
        print("  En Windows también necesitas poppler: https://github.com/oschwartz10612/poppler-windows/releases/")
        return 0
    
    if output_folder is None:
        output_folder = os.path.join(input_folder, "png_output")
    
    os.makedirs(output_folder, exist_ok=True)
    
    # Buscar todos los archivos PDF
    pdf_files = glob(os.path.join(input_folder, "*.pdf")) + \
                glob(os.path.join(input_folder, "*.PDF"))
    
    total_pages = 0
    
    for pdf_path in pdf_files:
        try:
            # Convertir PDF a lista de imágenes
            images = convert_from_path(pdf_path, dpi=dpi)
            
            filename_base = os.path.splitext(os.path.basename(pdf_path))[0]
            
            # Guardar cada página como PNG
            for i, image in enumerate(images, start=1):
                png_filename = f"{filename_base}_page_{i:03d}.png"
                png_path = os.path.join(output_folder, png_filename)
                image.save(png_path, 'PNG')
                total_pages += 1
            
            print(f"  ✓ Convertido: {os.path.basename(pdf_path)} ({len(images)} páginas)")
            
        except Exception as e:
            print(f"  ✗ Error convirtiendo {os.path.basename(pdf_path)}: {e}")
    
    print(f"\n✓ Conversión completada: {total_pages} páginas convertidas")
    return total_pages


# Ejemplo de uso (comentado - descomentar para usar):
# convert_pdf_to_png(r"datasets\mi_carpeta_pdf", r"datasets\mi_carpeta_png", dpi=200)

## 5. PASO 2: Preprocesamiento de Texto

Función que limpia y normaliza el texto extraído por OCR para mejorar la calidad de los features.

In [ ]:
# ============================================================================
# FUNCIÓN DE PREPROCESAMIENTO DE TEXTO
# ============================================================================

def preprocess_data(text):
    """
    Preprocesa el texto extraído por OCR aplicando técnicas de NLP
    
    Parámetros:
    -----------
    text : str
        Texto crudo extraído por OCR
    
    Retorna:
    --------
    str : Texto preprocesado y normalizado
    
    Justificación de cada paso:
    ----------------------------
    1. Lowercase: Normaliza el texto, reduce dimensionalidad del vocabulario
    2. Eliminación de saltos de línea y tabulaciones: Limpia formato OCR
    3. Eliminación de espacios múltiples: Normaliza espaciado
    4. Eliminación de números: Los números tienen poco valor semántico para clasificación
       de tipo de documento (el contenido conceptual importa más que valores específicos)
    5. Eliminación de puntuación: Reduce ruido, la estructura sintáctica es menos relevante
       que el vocabulario para este problema
    6. Tokenización: Divide texto en palabras individuales
    7. Eliminación de stopwords: Elimina palabras comunes sin valor discriminativo
       (the, is, at, which, on, etc.)
    8. Lemmatización: Reduce palabras a su forma base (running → run, better → good)
       - Preferimos lemmatización sobre stemming porque preserva palabras reales
       - Stemming sería más agresivo pero puede generar tokens sin significado
    
    Nota sobre la elección de técnicas:
    -----------------------------------
    - Para clasificación de documentos, el vocabulario técnico y específico de dominio
      es más importante que la estructura gramatical
    - La lemmatización preserva el significado mientras reduce variabilidad
    - Este preprocesamiento es estándar para problemas de clasificación de texto
    """
    
    # 1. Convertir a minúsculas
    text = text.lower()
    
    # 2. Eliminar saltos de línea y tabulaciones
    text = text.replace("\n", " ").replace("\t", " ")
    
    # 3. Eliminar espacios múltiples
    text = re.sub(r"\s+", " ", text)
    
    # 4. Eliminar números
    text = re.sub(r'\d+', '', text)
    
    # 5. Eliminar puntuación y caracteres especiales
    text = re.sub(r'[^\w\s]', '', text)
    
    # 6. Tokenización: dividir texto en palabras
    tokens = word_tokenize(text)
    
    # 7. Eliminar puntuación residual y stopwords
    data = [token for token in tokens if token not in punctuation]
    data = [token for token in data if token not in stopwords_list]
    
    # 8. Lemmatización: reducir palabras a su forma base
    lemmatizer = WordNetLemmatizer()
    final_text = []
    for token in data:
        word = lemmatizer.lemmatize(token)
        final_text.append(word)
    
    # Retornar texto procesado como string
    return " ".join(final_text)


# Probar la función con texto de ejemplo
example_text = "This is an EXAMPLE text with numbers 123 and punctuation!!!"
print("Texto original:", example_text)
print("Texto procesado:", preprocess_data(example_text))

## 6. PASO 2 (continuación): Extracción de Texto desde Imágenes

Función que procesa una carpeta de imágenes, extrae texto con OCR y crea un DataFrame.

In [ ]:
# ============================================================================
# FUNCIÓN DE CARGA Y EXTRACCIÓN DE TEXTO
# ============================================================================

def load_documents_from_images(dataset_path, class_labels_dict):
    """
    Carga imágenes de documentos, extrae texto con OCR y crea DataFrame
    
    Parámetros:
    -----------
    dataset_path : str
        Ruta a la carpeta principal del dataset
        Estructura esperada: dataset_path/clase1/imagen1.png
                            dataset_path/clase2/imagen1.png
    class_labels_dict : dict
        Diccionario mapeando nombre de clase a número {nombre: id}
        Ejemplo: {'email': 0, 'resume': 1, 'scientific_publication': 2}
    
    Retorna:
    --------
    pd.DataFrame : DataFrame con columnas ['Text', 'Label', 'Label_Name', 'Filename']
    
    Justificación de estructura:
    ----------------------------
    - Organizar imágenes por carpetas facilita el etiquetado automático
    - Incluir filename permite trazabilidad y debugging
    - Guardar tanto label numérico como nombre facilita análisis
    
    Manejo de errores:
    ------------------
    - Si una imagen falla en OCR, se registra pero no detiene el proceso
    - Esto hace el pipeline robusto ante imágenes corruptas o ilegibles
    """
    
    final_text = []
    final_label = []
    final_label_name = []
    final_filename = []
    
    # Obtener lista de carpetas (clases)
    image_folders = [f for f in os.listdir(dataset_path) 
                     if os.path.isdir(os.path.join(dataset_path, f))]
    
    print(f"Procesando {len(image_folders)} clases de documentos...")
    
    for label_name in image_folders:
        # Verificar que la clase esté en el diccionario
        if label_name not in class_labels_dict:
            print(f"  ⚠ Advertencia: '{label_name}' no está en class_labels_dict, omitiendo...")
            continue
        
        label_path = os.path.join(dataset_path, label_name)
        image_files = [f for f in os.listdir(label_path) 
                       if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tif', '.tiff'))]
        
        print(f"\n  Procesando clase '{label_name}': {len(image_files)} imágenes")
        
        for idx, filename in enumerate(image_files, 1):
            try:
                # Cargar imagen
                image_path = os.path.join(label_path, filename)
                image = Image.open(image_path)
                
                # Extraer texto con OCR
                text = pytesseract.image_to_string(image, lang='eng')
                
                # Preprocesar texto
                text_data = preprocess_data(text)
                
                # Validar que el texto no esté vacío
                if len(text_data.strip()) == 0:
                    print(f"    ⚠ Advertencia: {filename} generó texto vacío después de preprocesamiento")
                    continue
                
                # Almacenar datos
                final_text.append(text_data)
                final_label.append(class_labels_dict[label_name])
                final_label_name.append(label_name)
                final_filename.append(filename)
                
                if idx % 20 == 0:
                    print(f"    Progreso: {idx}/{len(image_files)} imágenes procesadas")
                
            except Exception as e:
                print(f"    ✗ Error procesando {filename}: {e}")
                continue
    
    # Crear DataFrame
    df = pd.DataFrame({
        'Text': final_text,
        'Label': final_label,
        'Label_Name': final_label_name,
        'Filename': final_filename
    })
    
    print(f"\n✓ Extracción completada: {len(df)} documentos procesados exitosamente")
    print(f"✓ Distribución de clases:")
    print(df['Label_Name'].value_counts())
    
    return df


# Ejemplo de uso (comentado - se ejecutará más adelante)
# class_labels = {'email': 0, 'resume': 1, 'scientific_publication': 2}
# df = load_documents_from_images(r"datasets\document-classification-dataset", class_labels)

## 7. Carga del Dataset Principal

Ejecutamos la carga de datos del dataset principal.

In [ ]:
# ============================================================================
# CARGA DEL DATASET
# ============================================================================

# Definir las clases y sus etiquetas numéricas
# Estas son las categorías de documentos que vamos a clasificar
class_labels = {
    'email': 0,
    'resume': 1,
    'scientific_publication': 2
}

# Ruta al dataset principal
DATASET_PATH = r"datasets\document-classification-dataset"

# Cargar y procesar el dataset
print("Iniciando carga del dataset...")
print("=" * 70)
df = load_documents_from_images(DATASET_PATH, class_labels)

# Mostrar información básica del dataset
print("\n" + "=" * 70)
print("INFORMACIÓN DEL DATASET CARGADO")
print("=" * 70)
print(f"Total de documentos: {len(df)}")
print(f"Columnas: {list(df.columns)}")
print(f"\nPrimeras filas del dataset:")
df.head()

## 8. PASO 3: Análisis Exploratorio Exhaustivo de Datos (EDA)

El análisis exploratorio es crucial para entender las características del dataset y tomar decisiones informadas sobre el modelado.

### Objetivos del EDA:
1. Verificar balance/desbalance de clases
2. Analizar distribución de longitud de textos
3. Identificar vocabulario más frecuente por clase
4. Detectar posibles problemas de calidad de datos
5. Visualizar características discriminativas entre clases

In [ ]:
# ============================================================================
# ANÁLISIS EXPLORATORIO: Información General del Dataset
# ============================================================================

def analyze_dataset_overview(df):
    """
    Muestra estadísticas generales del dataset
    
    Justificación:
    --------------
    - Entender el tamaño del dataset nos ayuda a elegir modelos apropiados
    - Dataset pequeño (<1000): modelos simples como Logistic Regression, Naive Bayes
    - Dataset grande (>10000): podemos usar modelos más complejos como ensemble methods
    """
    
    print("=" * 70)
    print("ANÁLISIS EXPLORATORIO DE DATOS (EDA)")
    print("=" * 70)
    
    print(f"\n1. DIMENSIONES DEL DATASET")
    print(f"   {'─' * 50}")
    print(f"   Total de documentos: {len(df)}")
    print(f"   Total de features: {df.shape[1]}")
    print(f"   Memoria utilizada: {df.memory_usage(deep=True).sum() / 1024:.2f} KB")
    
    print(f"\n2. TIPOS DE DATOS")
    print(f"   {'─' * 50}")
    print(df.dtypes)
    
    print(f"\n3. VALORES FALTANTES")
    print(f"   {'─' * 50}")
    missing = df.isnull().sum()
    if missing.sum() == 0:
        print("   ✓ No hay valores faltantes")
    else:
        print(missing[missing > 0])
    
    print(f"\n4. ESTADÍSTICAS DE LONGITUD DE TEXTO")
    print(f"   {'─' * 50}")
    df['text_length'] = df['Text'].apply(lambda x: len(x.split()))
    
    stats = df['text_length'].describe()
    print(f"   Promedio de palabras por documento: {stats['mean']:.2f}")
    print(f"   Mediana: {stats['50%']:.2f}")
    print(f"   Mínimo: {stats['min']:.0f} palabras")
    print(f"   Máximo: {stats['max']:.0f} palabras")
    print(f"   Desviación estándar: {stats['std']:.2f}")
    
    return df

# Ejecutar análisis general
df = analyze_dataset_overview(df)

In [ ]:
# ============================================================================
# ANÁLISIS EXPLORATORIO: Distribución de Clases
# ============================================================================

def analyze_class_distribution(df):
    """
    Analiza el balance de clases en el dataset
    
    Justificación:
    --------------
    - Dataset balanceado: accuracy es una buena métrica
    - Dataset desbalanceado: necesitamos usar F1-score, precision, recall
    - Desbalance severo (>10:1): considerar técnicas de resampling (SMOTE, undersampling)
    
    Criterio de balance:
    --------------------
    - Balanceado: diferencia <20% entre clases
    - Ligeramente desbalanceado: 20-50%
    - Moderadamente desbalanceado: 50-100%
    - Severamente desbalanceado: >100%
    """
    
    print(f"\n5. DISTRIBUCIÓN DE CLASES")
    print(f"   {'─' * 50}")
    
    class_counts = df['Label_Name'].value_counts()
    print(f"\n   Conteo absoluto:")
    for class_name, count in class_counts.items():
        percentage = (count / len(df)) * 100
        print(f"   {class_name:25s}: {count:4d} documentos ({percentage:5.2f}%)")
    
    # Calcular ratio de desbalance
    max_count = class_counts.max()
    min_count = class_counts.min()
    imbalance_ratio = max_count / min_count
    
    print(f"\n   Ratio de desbalance: {imbalance_ratio:.2f}:1")
    
    if imbalance_ratio < 1.2:
        balance_status = "✓ Dataset BALANCEADO"
        recommendation = "No se requieren técnicas de balanceo"
    elif imbalance_ratio < 1.5:
        balance_status = "⚠ Dataset LIGERAMENTE DESBALANCEADO"
        recommendation = "Considerar usar class_weight='balanced' en modelos"
    elif imbalance_ratio < 2.0:
        balance_status = "⚠ Dataset MODERADAMENTE DESBALANCEADO"
        recommendation = "Usar class_weight='balanced' y métricas como F1-score"
    else:
        balance_status = "✗ Dataset SEVERAMENTE DESBALANCEADO"
        recommendation = "Considerar SMOTE, undersampling o estratified sampling"
    
    print(f"\n   Estado: {balance_status}")
    print(f"   Recomendación: {recommendation}")
    
    return class_counts, imbalance_ratio

class_counts, imbalance_ratio = analyze_class_distribution(df)

In [ ]:
# ============================================================================
# VISUALIZACIÓN: Distribución de Clases
# ============================================================================

def plot_class_distribution(df):
    """
    Crea visualizaciones de la distribución de clases
    
    Justificación:
    --------------
    - Las visualizaciones facilitan identificar desbalances
    - Gráfico de barras: mejor para comparar cantidades exactas
    - Gráfico de pastel: mejor para ver proporciones relativas
    """
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Gráfico de barras
    class_counts = df['Label_Name'].value_counts()
    colors = sns.color_palette("husl", len(class_counts))
    
    axes[0].bar(class_counts.index, class_counts.values, color=colors, edgecolor='black', alpha=0.7)
    axes[0].set_xlabel('Clase de Documento', fontsize=12, fontweight='bold')
    axes[0].set_ylabel('Cantidad de Documentos', fontsize=12, fontweight='bold')
    axes[0].set_title('Distribución de Clases - Gráfico de Barras', fontsize=14, fontweight='bold')
    axes[0].grid(axis='y', alpha=0.3)
    
    # Añadir valores en las barras
    for i, (class_name, count) in enumerate(class_counts.items()):
        axes[0].text(i, count + max(class_counts) * 0.02, str(count), 
                     ha='center', va='bottom', fontsize=11, fontweight='bold')
    
    # Gráfico de pastel
    axes[1].pie(class_counts.values, labels=class_counts.index, autopct='%1.1f%%',
                colors=colors, startangle=90, explode=[0.05] * len(class_counts))
    axes[1].set_title('Distribución de Clases - Proporción', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

plot_class_distribution(df)

In [ ]:
# ============================================================================
# VISUALIZACIÓN: Distribución de Longitud de Texto por Clase
# ============================================================================

def plot_text_length_distribution(df):
    """
    Analiza la distribución de longitud de texto por clase
    
    Justificación:
    --------------
    - Si las clases tienen longitudes características diferentes, esto es un feature útil
    - Por ejemplo: emails tienden a ser más cortos que publicaciones científicas
    - Esta información puede ser usada como feature adicional en el modelo
    """
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # 1. Boxplot de longitud por clase
    df.boxplot(column='text_length', by='Label_Name', ax=axes[0, 0])
    axes[0, 0].set_title('Distribución de Longitud de Texto por Clase', fontsize=12, fontweight='bold')
    axes[0, 0].set_xlabel('Clase de Documento', fontsize=11)
    axes[0, 0].set_ylabel('Número de Palabras', fontsize=11)
    plt.sca(axes[0, 0])
    plt.xticks(rotation=15)
    
    # 2. Histograma superpuesto
    for label_name in df['Label_Name'].unique():
        subset = df[df['Label_Name'] == label_name]['text_length']
        axes[0, 1].hist(subset, alpha=0.5, label=label_name, bins=20, edgecolor='black')
    
    axes[0, 1].set_xlabel('Número de Palabras', fontsize=11)
    axes[0, 1].set_ylabel('Frecuencia', fontsize=11)
    axes[0, 1].set_title('Histograma de Longitud de Texto', fontsize=12, fontweight='bold')
    axes[0, 1].legend()
    axes[0, 1].grid(alpha=0.3)
    
    # 3. Violin plot
    import seaborn as sns
    sns.violinplot(data=df, x='Label_Name', y='text_length', ax=axes[1, 0])
    axes[1, 0].set_title('Violin Plot - Distribución de Longitud', fontsize=12, fontweight='bold')
    axes[1, 0].set_xlabel('Clase de Documento', fontsize=11)
    axes[1, 0].set_ylabel('Número de Palabras', fontsize=11)
    axes[1, 0].tick_params(axis='x', rotation=15)
    
    # 4. Estadísticas por clase
    stats_by_class = df.groupby('Label_Name')['text_length'].describe()[['mean', 'std', 'min', 'max']]
    
    axes[1, 1].axis('off')
    table_data = []
    for idx, row in stats_by_class.iterrows():
        table_data.append([idx, f"{row['mean']:.1f}", f"{row['std']:.1f}", 
                          f"{row['min']:.0f}", f"{row['max']:.0f}"])
    
    table = axes[1, 1].table(cellText=table_data, 
                             colLabels=['Clase', 'Media', 'Std', 'Min', 'Max'],
                             cellLoc='center', loc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 2)
    axes[1, 1].set_title('Estadísticas de Longitud por Clase', fontsize=12, fontweight='bold', pad=20)
    
    plt.tight_layout()
    plt.show()
    
    print("\nANÁLISIS DE LONGITUD DE TEXTO POR CLASE:")
    print("─" * 70)
    for label_name in df['Label_Name'].unique():
        subset = df[df['Label_Name'] == label_name]['text_length']
        print(f"\n{label_name}:")
        print(f"  Promedio: {subset.mean():.2f} palabras")
        print(f"  Rango: {subset.min():.0f} - {subset.max():.0f} palabras")

plot_text_length_distribution(df)

In [ ]:
# ============================================================================
# ANÁLISIS: Vocabulario y Palabras Más Frecuentes
# ============================================================================

def analyze_vocabulary(df, top_n=20):
    """
    Analiza el vocabulario más frecuente por clase
    
    Justificación:
    --------------
    - Identificar palabras discriminativas ayuda a entender qué aprenderá el modelo
    - Si hay mucho solapamiento de vocabulario, el problema es más difícil
    - Palabras únicas por clase son buenos indicadores de esa clase
    
    Parámetros:
    -----------
    top_n : int
        Número de palabras más frecuentes a mostrar por clase
    """
    
    from collections import Counter
    
    print(f"\nANÁLISIS DE VOCABULARIO (Top {top_n} palabras por clase)")
    print("=" * 70)
    
    vocab_by_class = {}
    
    for label_name in df['Label_Name'].unique():
        # Obtener todos los textos de esta clase
        class_texts = df[df['Label_Name'] == label_name]['Text']
        
        # Combinar todos los textos y contar palabras
        all_words = []
        for text in class_texts:
            all_words.extend(text.split())
        
        # Contar frecuencias
        word_counts = Counter(all_words)
        vocab_by_class[label_name] = word_counts
        
        # Mostrar top palabras
        print(f"\n{label_name.upper()}:")
        print(f"{'─' * 50}")
        print(f"  Vocabulario total: {len(word_counts)} palabras únicas")
        print(f"  Total de palabras: {sum(word_counts.values())}")
        print(f"\n  Top {top_n} palabras más frecuentes:")
        
        for i, (word, count) in enumerate(word_counts.most_common(top_n), 1):
            print(f"    {i:2d}. {word:20s}: {count:4d} veces")
    
    return vocab_by_class

vocab_by_class = analyze_vocabulary(df, top_n=15)

In [ ]:
# ============================================================================
# VISUALIZACIÓN: Word Clouds por Clase
# ============================================================================

def plot_wordclouds(df):
    """
    Genera word clouds para cada clase de documento
    
    Justificación:
    --------------
    - Word clouds permiten visualizar rápidamente el vocabulario característico
    - Palabras grandes = más frecuentes en esa clase
    - Ayuda a validar que el OCR y preprocesamiento funcionan correctamente
    """
    
    classes = df['Label_Name'].unique()
    n_classes = len(classes)
    
    fig, axes = plt.subplots(1, n_classes, figsize=(18, 5))
    
    if n_classes == 1:
        axes = [axes]
    
    for idx, label_name in enumerate(classes):
        # Obtener todos los textos de esta clase
        class_texts = ' '.join(df[df['Label_Name'] == label_name]['Text'])
        
        # Generar word cloud
        wordcloud = WordCloud(width=800, height=400, 
                             background_color='white',
                             colormap='viridis',
                             max_words=100,
                             relative_scaling=0.5,
                             min_font_size=10).generate(class_texts)
        
        # Mostrar
        axes[idx].imshow(wordcloud, interpolation='bilinear')
        axes[idx].set_title(f'Word Cloud: {label_name}', fontsize=14, fontweight='bold')
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()

plot_wordclouds(df)

## 9. PASO 4: División Estratificada de Datos (70-20-10)

División del dataset en conjuntos de entrenamiento, validación y prueba.

### Justificación de la División 70-20-10:
- **70% Entrenamiento**: Suficiente datos para que el modelo aprenda patrones
- **20% Validación**: Para ajustar hiperparámetros y evitar overfitting
- **10% Test**: Evaluación final del modelo con datos nunca vistos

### Estratificación:
- Mantenemos la proporción de clases en cada conjunto
- Crítico para datasets desbalanceados
- Asegura representatividad en train/val/test

In [ ]:
# ============================================================================
# DIVISIÓN ESTRATIFICADA DEL DATASET
# ============================================================================

def split_dataset_stratified(df, train_size=0.7, val_size=0.2, test_size=0.1, random_state=42):
    """
    Divide el dataset en train, validation y test de forma estratificada
    
    Parámetros:
    -----------
    df : pd.DataFrame
        Dataset completo
    train_size : float
        Proporción para entrenamiento (default: 0.7)
    val_size : float
        Proporción para validación (default: 0.2)
    test_size : float
        Proporción para test (default: 0.1)
    random_state : int
        Semilla para reproducibilidad
    
    Retorna:
    --------
    tuple : (X_train, X_val, X_test, y_train, y_val, y_test)
    
    Justificación:
    --------------
    - Estratificación mantiene proporciones de clases en todos los conjuntos
    - Random state asegura reproducibilidad de experimentos
    - División en 3 conjuntos permite validación adecuada sin contaminar test set
    
    Proceso:
    --------
    1. Primero dividimos en train+val (90%) y test (10%)
    2. Luego dividimos train+val en train (70%) y val (20%)
    3. Esto garantiza las proporciones 70-20-10 del total
    """
    
    # Verificar que las proporciones sumen 1.0
    assert abs(train_size + val_size + test_size - 1.0) < 0.001, \
        f"Las proporciones deben sumar 1.0 (actual: {train_size + val_size + test_size})"
    
    print("=" * 70)
    print("DIVISIÓN ESTRATIFICADA DEL DATASET")
    print("=" * 70)
    
    # Extraer features (X) y labels (y)
    X = df['Text'].values
    y = df['Label'].values
    y_names = df['Label_Name'].values
    
    # Paso 1: Separar test set (10%)
    test_proportion = test_size
    X_temp, X_test, y_temp, y_test = train_test_split(
        X, y, 
        test_size=test_proportion,
        stratify=y,
        random_state=random_state
    )
    
    # Paso 2: Separar train y validation del resto
    # val_size_adjusted es la proporción de validación respecto al conjunto temporal
    val_size_adjusted = val_size / (train_size + val_size)
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp,
        test_size=val_size_adjusted,
        stratify=y_temp,
        random_state=random_state
    )
    
    # Mostrar estadísticas
    print(f"\nTamaño total del dataset: {len(df)} documentos")
    print(f"\n{'Conjunto':<15} {'Documentos':>12} {'Proporción':>12}")
    print("─" * 45)
    print(f"{'Entrenamiento':<15} {len(X_train):>12} {len(X_train)/len(df)*100:>11.1f}%")
    print(f"{'Validación':<15} {len(X_val):>12} {len(X_val)/len(df)*100:>11.1f}%")
    print(f"{'Test':<15} {len(X_test):>12} {len(X_test)/len(df)*100:>11.1f}%")
    
    # Verificar estratificación
    print(f"\n{'Distribución de clases en cada conjunto:':}")
    print("─" * 70)
    
    class_names = {v: k for k, v in class_labels.items()}
    
    for split_name, y_split in [('Train', y_train), ('Validation', y_val), ('Test', y_test)]:
        print(f"\n{split_name}:")
        unique, counts = np.unique(y_split, return_counts=True)
        for label, count in zip(unique, counts):
            percentage = (count / len(y_split)) * 100
            print(f"  {class_names[label]:25s}: {count:3d} ({percentage:5.2f}%)")
    
    print("\n✓ División completada exitosamente")
    print("✓ Estratificación verificada: proporciones de clases mantenidas")
    
    return X_train, X_val, X_test, y_train, y_val, y_test


# Ejecutar división del dataset
X_train, X_val, X_test, y_train, y_val, y_test = split_dataset_stratified(
    df, 
    train_size=0.7, 
    val_size=0.2, 
    test_size=0.1,
    random_state=42
)

## 10. Feature Engineering: TF-IDF Vectorization

Convertimos el texto a representación numérica usando TF-IDF.

### Justificación de TF-IDF:
- **TF (Term Frequency)**: Mide qué tan frecuente es una palabra en un documento
- **IDF (Inverse Document Frequency)**: Penaliza palabras que aparecen en muchos documentos
- **TF-IDF = TF × IDF**: Resalta palabras importantes pero no comunes

### Por qué TF-IDF para este problema:
1. **Eficaz para clasificación de texto**: Captura importancia relativa de palabras
2. **Reduce peso de palabras comunes**: Automáticamente maneja palabras frecuentes
3. **Sparse pero eficiente**: Matrices sparse ahorran memoria
4. **Baseline sólido**: Estado del arte para muchos problemas de NLP

### Alternativas consideradas:
- **Bag of Words (Count)**: Más simple pero ignora importancia relativa
- **Word Embeddings (Word2Vec, GloVe)**: Capturan semántica pero requieren más datos
- **BERT/Transformers**: Mejor rendimiento pero computacionalmente costoso

### Configuración de TF-IDF:
- **ngram_range=(1,2)**: Incluye palabras individuales y bigramas
  - Unigrama: "machine learning" → ["machine", "learning"]
  - Bigrama: "machine learning" → ["machine", "learning", "machine learning"]
  - Bigramas capturan contexto local y frases específicas de dominio
- **max_features**: Limitamos dimensionalidad para evitar overfitting
- **min_df**: Ignoramos palabras muy raras (ruido)
- **max_df**: Ignoramos palabras muy comunes (poco discriminativas)

In [ ]:
# ============================================================================
# FEATURE ENGINEERING: TF-IDF VECTORIZATION
# ============================================================================

def create_tfidf_features(X_train, X_val, X_test, 
                          ngram_range=(1, 2), 
                          max_features=5000,
                          min_df=2,
                          max_df=0.95):
    """
    Convierte texto a features TF-IDF
    
    Parámetros:
    -----------
    X_train, X_val, X_test : array-like
        Conjuntos de texto
    ngram_range : tuple
        Rango de n-gramas a considerar (default: unigrams + bigrams)
    max_features : int
        Número máximo de features a extraer
    min_df : int o float
        Frecuencia mínima de documento (ignora términos muy raros)
    max_df : float
        Frecuencia máxima de documento (ignora términos muy comunes)
    
    Retorna:
    --------
    tuple : (X_train_tfidf, X_val_tfidf, X_test_tfidf, vectorizer)
    
    Nota importante:
    ----------------
    - SOLO entrenamos el vectorizer con X_train (fit)
    - Aplicamos la transformación a val y test (transform)
    - Esto previene data leakage del conjunto de test
    """
    
    print("=" * 70)
    print("FEATURE ENGINEERING: TF-IDF VECTORIZATION")
    print("=" * 70)
    
    # Crear vectorizador TF-IDF
    vectorizer = TfidfVectorizer(
        ngram_range=ngram_range,
        max_features=max_features,
        min_df=min_df,
        max_df=max_df,
        sublinear_tf=True,  # Usa escala logarítmica para TF
        use_idf=True
    )
    
    # Entrenar SOLO con datos de entrenamiento
    print(f"\nEntrenando TF-IDF vectorizer...")
    print(f"  Configuración:")
    print(f"    - N-gram range: {ngram_range}")
    print(f"    - Max features: {max_features}")
    print(f"    - Min document frequency: {min_df}")
    print(f"    - Max document frequency: {max_df}")
    
    X_train_tfidf = vectorizer.fit_transform(X_train)
    
    # Transformar validation y test
    X_val_tfidf = vectorizer.transform(X_val)
    X_test_tfidf = vectorizer.transform(X_test)
    
    # Mostrar estadísticas
    print(f"\n✓ Vectorización completada")
    print(f"\n  Estadísticas de features:")
    print(f"    - Vocabulario total: {len(vectorizer.vocabulary_)} términos")
    print(f"    - Features generados: {X_train_tfidf.shape[1]}")
    print(f"\n  Dimensiones de matrices:")
    print(f"    - Train:      {X_train_tfidf.shape} (documentos × features)")
    print(f"    - Validation: {X_val_tfidf.shape}")
    print(f"    - Test:       {X_test_tfidf.shape}")
    print(f"\n  Sparsity (% de valores cero):")
    print(f"    - Train:      {(1.0 - X_train_tfidf.nnz / (X_train_tfidf.shape[0] * X_train_tfidf.shape[1])) * 100:.2f}%")
    
    # Mostrar algunos features importantes
    feature_names = vectorizer.get_feature_names_out()
    print(f"\n  Ejemplos de features extraídos:")
    print(f"    Primeros 10: {list(feature_names[:10])}")
    print(f"    Últimos 10:  {list(feature_names[-10:])}")
    
    return X_train_tfidf, X_val_tfidf, X_test_tfidf, vectorizer


# Crear features TF-IDF
X_train_tfidf, X_val_tfidf, X_test_tfidf, tfidf_vectorizer = create_tfidf_features(
    X_train, X_val, X_test,
    ngram_range=(1, 2),
    max_features=5000,
    min_df=2,
    max_df=0.95
)

## 11. Análisis de PCA (Principal Component Analysis)

Evaluamos si PCA es apropiado para este problema.

### ¿Qué es PCA?
- Técnica de reducción de dimensionalidad
- Encuentra direcciones de máxima varianza en los datos
- Proyecta datos a espacio de menor dimensión

### Criterios para aplicar PCA:
1. **Alta dimensionalidad**: TF-IDF genera muchas features (5000+)
2. **Features correlacionados**: PCA es útil si hay redundancia
3. **Reducir overfitting**: Menos features = menor riesgo de overfitting
4. **Visualización**: PCA permite visualizar datos en 2D/3D

### Desventajas de PCA para NLP:
1. **Pérdida de interpretabilidad**: Componentes principales no son palabras
2. **TF-IDF ya es sparse**: PCA genera matrices densas (más memoria)
3. **Puede perder información discriminativa**: Features raros pero importantes

### Decisión:
Evaluaremos si PCA mejora el rendimiento, pero anticipamos que para clasificación
de documentos con TF-IDF, mantener features originales suele ser mejor.

In [ ]:
# ============================================================================
# ANÁLISIS DE PCA: ¿ES NECESARIO?
# ============================================================================

def analyze_pca_necessity(X_train_tfidf, y_train, n_components=0.95):
    """
    Analiza si PCA es beneficioso para este problema
    
    Parámetros:
    -----------
    X_train_tfidf : sparse matrix
        Features TF-IDF de entrenamiento
    y_train : array
        Labels de entrenamiento
    n_components : float o int
        Si float (0-1): mantiene ese % de varianza
        Si int: mantiene ese número de componentes
    
    Retorna:
    --------
    dict : Estadísticas de PCA para toma de decisión
    """
    
    print("=" * 70)
    print("ANÁLISIS DE PCA (Principal Component Analysis)")
    print("=" * 70)
    
    print(f"\nDimensionalidad actual: {X_train_tfidf.shape[1]} features")
    print(f"Sparsity: {(1.0 - X_train_tfidf.nnz / (X_train_tfidf.shape[0] * X_train_tfidf.shape[1])) * 100:.2f}%")
    
    # Convertir a array denso para PCA (solo para análisis)
    print(f"\nConvirtiendo matriz sparse a densa para análisis PCA...")
    print(f"⚠ Advertencia: Esto puede consumir mucha memoria")
    
    try:
        X_dense = X_train_tfidf.toarray()
        
        # Aplicar PCA
        pca = PCA(n_components=n_components, random_state=42)
        X_pca = pca.fit_transform(X_dense)
        
        # Estadísticas
        print(f"\n✓ PCA completado")
        print(f"\nResultados:")
        print(f"  Componentes mantenidos: {pca.n_components_}")
        print(f"  Varianza explicada: {pca.explained_variance_ratio_.sum()*100:.2f}%")
        print(f"  Reducción de dimensionalidad: {X_train_tfidf.shape[1]} → {pca.n_components_}")
        print(f"  Factor de reducción: {X_train_tfidf.shape[1] / pca.n_components_:.2f}x")
        
        # Varianza por componente
        print(f"\n  Varianza explicada por los primeros 10 componentes:")
        for i in range(min(10, len(pca.explained_variance_ratio_))):
            print(f"    PC{i+1}: {pca.explained_variance_ratio_[i]*100:.2f}%")
        
        # Visualizar varianza acumulada
        plt.figure(figsize=(10, 5))
        cumsum = np.cumsum(pca.explained_variance_ratio_)
        plt.plot(range(1, len(cumsum)+1), cumsum, 'bo-', linewidth=2, markersize=4)
        plt.xlabel('Número de Componentes', fontsize=12)
        plt.ylabel('Varianza Explicada Acumulada', fontsize=12)
        plt.title('Curva de Varianza Acumulada de PCA', fontsize=14, fontweight='bold')
        plt.grid(True, alpha=0.3)
        plt.axhline(y=0.95, color='r', linestyle='--', label='95% varianza')
        plt.axhline(y=0.90, color='orange', linestyle='--', label='90% varianza')
        plt.legend()
        plt.tight_layout()
        plt.show()
        
        # Decisión sobre PCA
        print(f"\n{'DECISIÓN SOBRE EL USO DE PCA:':}")
        print("─" * 70)
        
        if pca.n_components_ < X_train_tfidf.shape[1] * 0.3:
            print(f"✓ RECOMENDACIÓN: USAR PCA")
            print(f"  Razón: Reducción significativa ({X_train_tfidf.shape[1] / pca.n_components_:.1f}x) manteniendo 95% varianza")
            recommendation = True
        else:
            print(f"✗ RECOMENDACIÓN: NO USAR PCA")
            print(f"  Razón: Se requieren {pca.n_components_} componentes (>{X_train_tfidf.shape[1] * 0.3:.0f})")
            print(f"  - La reducción no es suficientemente significativa")
            print(f"  - TF-IDF sparse es más eficiente en memoria")
            print(f"  - Mantenemos interpretabilidad de features")
            recommendation = False
        
        return {
            'pca': pca,
            'n_components': pca.n_components_,
            'variance_explained': pca.explained_variance_ratio_.sum(),
            'X_pca': X_pca,
            'recommendation': recommendation
        }
        
    except MemoryError:
        print(f"\n✗ Error: Memoria insuficiente para PCA completo")
        print(f"\nDECISIÓN: NO USAR PCA")
        print(f"  - Dataset demasiado grande para PCA")
        print(f"  - Continuaremos con TF-IDF sparse")
        return {'recommendation': False}


# Analizar PCA
pca_analysis = analyze_pca_necessity(X_train_tfidf, y_train, n_components=0.95)

## 12. PASO 5: Entrenamiento de Modelos con Validación Cruzada

Entrenaremos múltiples algoritmos y seleccionaremos el mejor mediante validación cruzada.

### Algoritmos Seleccionados y Justificación:

#### 1. **Logistic Regression**
- **Pros**: Rápido, interpretable, funciona bien con features sparse
- **Cons**: Asume separación lineal
- **Uso**: Baseline excelente para clasificación de texto

#### 2. **Multinomial Naive Bayes**
- **Pros**: Diseñado específicamente para datos de conteo (TF-IDF)
- **Pros**: Muy rápido, funciona bien con poco datos
- **Cons**: Asume independencia de features (raramente cierto)
- **Uso**: Estado del arte clásico para clasificación de texto

#### 3. **Linear SVM (LinearSVC)**
- **Pros**: Encuentra hiperplano de máxima separación, robusto
- **Pros**: Eficiente con datos high-dimensional sparse
- **Cons**: Sensible a escala, requiere tuning de C
- **Uso**: Muy efectivo para clasificación de documentos

#### 4. **Random Forest**
- **Pros**: Maneja relaciones no lineales, robusto a overfitting
- **Pros**: Importancia de features interpretable
- **Cons**: Puede ser lento con muchos features, no optimizado para sparse
- **Uso**: Ensemble method robusto

#### 5. **Gradient Boosting (LightGBM)**
- **Pros**: Estado del arte para muchos problemas, maneja no-linealidad
- **Pros**: LightGBM es rápido y eficiente con memoria
- **Cons**: Requiere tuning cuidadoso, riesgo de overfitting
- **Uso**: Potencialmente el mejor rendimiento

### Validación Cruzada Estratificada (5-fold):
- Divide datos en 5 partes
- Entrena 5 veces, cada vez usando 4 partes para train y 1 para validation
- Promedia resultados para estimación robusta
- **Detecta overfitting**: Si training score >> validation score → overfitting

In [ ]:
# ============================================================================
# CONFIGURACIÓN DE MODELOS Y CROSS-VALIDATION
# ============================================================================

def train_and_evaluate_models(X_train, X_val, y_train, y_val, cv_folds=5):
    """
    Entrena múltiples modelos y evalúa con validación cruzada
    
    Parámetros:
    -----------
    X_train : sparse matrix
        Features de entrenamiento
    X_val : sparse matrix
        Features de validación
    y_train : array
        Labels de entrenamiento
    y_val : array
        Labels de validación
    cv_folds : int
        Número de folds para cross-validation
    
    Retorna:
    --------
    dict : Resultados de todos los modelos
    """
    
    print("=" * 70)
    print("ENTRENAMIENTO Y EVALUACIÓN DE MODELOS")
    print("=" * 70)
    
    # Definir modelos a entrenar
    # Cada modelo incluye justificación de hiperparámetros
    models = {
        'Logistic Regression': LogisticRegression(
            max_iter=1000,
            C=1.0,  # Regularización L2, C alto = menos regularización
            class_weight='balanced',  # Maneja desbalance de clases
            random_state=42,
            solver='liblinear'  # Eficiente para datasets pequeños-medianos
        ),
        
        'Naive Bayes': MultinomialNB(
            alpha=0.1  # Suavizado de Laplace, previene probabilidades cero
        ),
        
        'Linear SVM': LinearSVC(
            C=1.0,  # Parámetro de regularización
            class_weight='balanced',
            max_iter=1000,
            random_state=42,
            dual=False  # False es más eficiente cuando n_samples > n_features
        ),
        
        'Random Forest': RandomForestClassifier(
            n_estimators=100,  # 100 árboles
            max_depth=None,  # Sin límite de profundidad
            min_samples_split=5,  # Mínimo de muestras para dividir nodo
            min_samples_leaf=2,  # Mínimo de muestras en hoja
            class_weight='balanced',
            random_state=42,
            n_jobs=-1  # Usar todos los cores CPU
        ),
        
        'LightGBM': LGBMClassifier(
            n_estimators=100,
            max_depth=7,
            learning_rate=0.1,
            num_leaves=31,
            class_weight='balanced',
            random_state=42,
            verbose=-1
        )
    }
    
    results = {}
    best_model = None
    best_score = 0
    
    # Configurar validación cruzada estratificada
    skf = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)
    
    print(f"\nConfiguración de validación cruzada:")
    print(f"  - Número de folds: {cv_folds}")
    print(f"  - Estratificación: Sí (mantiene proporción de clases)")
    print(f"  - Métrica principal: Accuracy")
    print(f"  - Métricas adicionales: F1-score (weighted)")
    
    # Entrenar cada modelo
    for model_name, model in models.items():
        print(f"\n{'═' * 70}")
        print(f"Entrenando: {model_name}")
        print(f"{'═' * 70}")
        
        try:
            # Cross-validation
            print(f"  Ejecutando {cv_folds}-fold cross-validation...")
            
            # Convertir a array denso solo para Random Forest y LightGBM
            if model_name in ['Random Forest', 'LightGBM']:
                X_train_array = X_train.toarray()
                X_val_array = X_val.toarray()
            else:
                X_train_array = X_train
                X_val_array = X_val
            
            # Realizar cross-validation
            cv_scores = cross_val_score(
                model, X_train_array, y_train,
                cv=skf, scoring='accuracy', n_jobs=-1
            )
            
            # Entrenar en todo el conjunto de entrenamiento
            print(f"  Entrenando en dataset completo de entrenamiento...")
            model.fit(X_train_array, y_train)
            
            # Evaluar en validation set
            y_val_pred = model.predict(X_val_array)
            y_train_pred = model.predict(X_train_array)
            
            # Calcular métricas
            train_accuracy = accuracy_score(y_train, y_train_pred)
            val_accuracy = accuracy_score(y_val, y_val_pred)
            cv_mean = cv_scores.mean()
            cv_std = cv_scores.std()
            
            # Calcular F1-score
            precision, recall, f1, _ = precision_recall_fscore_support(
                y_val, y_val_pred, average='weighted'
            )
            
            # Detectar overfitting
            overfitting = train_accuracy - val_accuracy
            
            # Mostrar resultados
            print(f"\n  {'RESULTADOS':}")
            print(f"    {'─' * 50}")
            print(f"    Cross-Validation Accuracy: {cv_mean:.4f} ± {cv_std:.4f}")
            print(f"    Training Accuracy:         {train_accuracy:.4f}")
            print(f"    Validation Accuracy:       {val_accuracy:.4f}")
            print(f"    Validation F1-Score:       {f1:.4f}")
            print(f"    Validation Precision:      {precision:.4f}")
            print(f"    Validation Recall:         {recall:.4f}")
            print(f"\n    {'Análisis de Overfitting':}")
            print(f"    Diferencia Train-Val:      {overfitting:.4f}")
            
            if overfitting > 0.15:
                print(f"    ⚠ OVERFITTING DETECTADO (diferencia > 0.15)")
            elif overfitting > 0.05:
                print(f"    ⚠ Posible overfitting leve (diferencia > 0.05)")
            else:
                print(f"    ✓ Sin overfitting significativo")
            
            # Guardar resultados
            results[model_name] = {
                'model': model,
                'cv_scores': cv_scores,
                'cv_mean': cv_mean,
                'cv_std': cv_std,
                'train_accuracy': train_accuracy,
                'val_accuracy': val_accuracy,
                'f1_score': f1,
                'precision': precision,
                'recall': recall,
                'overfitting': overfitting,
                'y_val_pred': y_val_pred
            }
            
            # Actualizar mejor modelo
            if val_accuracy > best_score:
                best_score = val_accuracy
                best_model = model_name
            
            print(f"  ✓ {model_name} completado exitosamente")
            
        except Exception as e:
            print(f"  ✗ Error entrenando {model_name}: {e}")
            continue
    
    # Resumen final
    print(f"\n{'═' * 70}")
    print(f"RESUMEN DE RESULTADOS")
    print(f"{'═' * 70}")
    
    # Crear tabla comparativa
    print(f"\n{'Modelo':<20} {'CV Accuracy':<15} {'Val Accuracy':<15} {'F1-Score':<12} {'Overfitting':<12}")
    print(f"{'─' * 80}")
    
    for model_name, res in sorted(results.items(), key=lambda x: x[1]['val_accuracy'], reverse=True):
        print(f"{model_name:<20} {res['cv_mean']:.4f} ± {res['cv_std']:.3f}   "
              f"{res['val_accuracy']:.4f}          "
              f"{res['f1_score']:.4f}       "
              f"{res['overfitting']:+.4f}")
    
    print(f"\n{'✓ MEJOR MODELO:'} {best_model} (Validation Accuracy: {best_score:.4f})")
    
    return results, best_model


# Entrenar modelos
print("Iniciando entrenamiento de modelos...")
print("Esto puede tomar varios minutos dependiendo del tamaño del dataset...\n")

model_results, best_model_name = train_and_evaluate_models(
    X_train_tfidf, X_val_tfidf, y_train, y_val, cv_folds=5
)